In [1]:
import requests
import json
import time

In [2]:
api_key = 'REDACTED'
api_username = 'REDACTED'

In [3]:
start = 251034
install_count = 0

In [4]:
#help from https://meta.discourse.org/t/discourse-api-docs-mention-a-request-body-for-a-get-request/231137/13
def get_post_before(start):
    endpoint = f"https://forum.image.sc/posts.json?before={start}"
    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key,
        "Api-Username": api_username,
    }
    response = requests.get(endpoint, headers=headers) 
    if response.status_code == 200:
        return response.json()
    
    

In [5]:
test = get_post_before(start)

In [6]:
post_list = []
while len(post_list)<2000:
    posts = get_post_before(start)
    post_list += [x['cooked'] for x in posts['latest_posts']]
    print(len(post_list))
    start -= 50
    

41
74
106
139
176
218
258
289
332
374
412
442
480
520
558
595
638
675
708
752
785
813
856
892
928
972
1007
1038
1082
1123
1142
1172
1194
1234
1264
1303
1342
1381
1416
1443
1469
1502
1539
1577
1611
1650
1683
1719
1755
1791
1823
1861
1885
1914
1945
1976
2008


In [7]:
len([x for x in post_list if 'install' in x.lower()])

146

In [8]:
with open('2008_forum_posts.txt','w') as postfile:
    postfile.write(' ;;;; '.join(post_list))

In [9]:
14600/2008

7.270916334661354

In [12]:
def get_topic_ids(page):
    endpoint = f"https://forum.image.sc/latest.json?page={page}"
    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key,
        "Api-Username": api_username,
    }
    response = requests.get(endpoint, headers=headers) 
    if response.status_code == 200:
        page_contents = response.json()
        return [x['id'] for x in page_contents["topic_list"]["topics"]]
    else:
        print(f"Non 200 response code for page {page} (response code was {response.status_code}). Sleeping 3 seconds")
        time.sleep(3)
        return False
    

In [13]:
latest_topics = []
page = 1
while len(latest_topics)<=500:
    latest_topics += get_topic_ids(page)
    latest_topics = list(set(latest_topics))
    page += 1
    print(len(latest_topics))

30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510


In [16]:
def get_all_post_text(topic):
    endpoint = f"https://forum.image.sc/t/{topic}.json"
    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key,
        "Api-Username": api_username,
    }
    response = requests.get(endpoint, headers=headers) 
    if response.status_code == 200:
        page_contents = response.json()
        return [x['cooked'] for x in page_contents["post_stream"]["posts"]]
    else:
        print(f"Non 200 response code for topic {topic} (response code was {response.status_code}). Sleeping 30 seconds")
        time.sleep(30)
        return False

In [17]:
topic_post_dict = {}
for eachtopic in latest_topics:
    all_posts = get_all_post_text(eachtopic)
    if all_posts:
        topic_post_dict[eachtopic]=all_posts

Non 200 response code for topic 45459 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78227 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 55841 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78373 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 55987 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 55991 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78640 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78646 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78764 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78766 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78887 (response code was 429). Sleeping 30 seconds
Non 200 response code for topic 78888 (response code was 429). Sleeping 30 seconds
Non 

In [18]:
with open("500ish_forum_topics.json",'w') as topic_post_file:
    json.dump(topic_post_dict,topic_post_file)

In [21]:
topic_count = 0
post_count = 0
topics_with_install_count = 0
for value in list(topic_post_dict.values()):
    topic_count +=1
    post_count += len(value)
    if 'install' in ' '.join(value).lower():
        topics_with_install_count +=1
print(topic_count,post_count,topics_with_install_count)
    

494 2604 96


In [22]:
9600/494

19.4331983805668

In [23]:
topic_count = 0
post_count = 0
topics_with_install_in_top_post_count = 0
for value in list(topic_post_dict.values()):
    topic_count +=1
    post_count += len(value)
    if 'install' in value[0]:
        topics_with_install_in_top_post_count +=1
print(topic_count,post_count,topics_with_install_in_top_post_count)

494 2604 49


In [24]:
4900/494

9.919028340080972

In [25]:
topic_count = 0
post_count = 0
posts_with_install_count = 0
for value in list(topic_post_dict.values()):
    topic_count +=1
    post_count += len(value)
    for eachpost in value:
        if 'install' in eachpost:
            posts_with_install_count +=1
print(topic_count,post_count,posts_with_install_count)

494 2604 208


In [26]:
20800/2604

7.987711213517665